In [1]:
llm_config={"model": "gpt-4o-mini"}

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI is pioneering the future of artificial intelligence education. Founded by Andrew Ng, it offers comprehensive courses designed to empower learners at all levels. With a focus on practical skills and real-world applications, the platform helps individuals and organizations harness the power of deep learning. Through hands-on projects and a vibrant community, students build expertise to tackle complex problems. As AI continues to reshape industries, DeepLearning.AI equips the next generation of innovators, making cutting-edge technology accessible to all. Whether you're a beginner or an advanced practitioner, DeepLearning.AI is your gateway to a smarter future.


In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI is pioneering the future of artificial intelligence education. Founded by Andrew Ng, it offers comprehensive courses designed to empower learners at all levels. With a focus on practical skills and real-world applications, the platform helps individuals and organizations harness the power of deep learning. Through hands-on projects and a vibrant community, students build expertise to tackle complex problems. As AI continues to reshape industries, DeepLearning.AI equips the next generation of innovators, making cutting-edge technology accessible to all. Whether you're a beginner or an advanced practitioner, DeepLearning.AI is your gateway to a smarter future.

-------

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI is pioneering the future of artificial intelligence education. Founded by Andrew Ng, it offers comprehensive courses designed to empower learners at all levels. With a focus on practical skills and real-world applications, the platform helps individuals and organizations harness the power of deep learning. Through hands-on projects and a vibrant community, students build expertise to tackle complex problems. As AI continues to reshape industries, DeepLearning.AI equips the next generation of innovators, making cutting-edge technology accessible to all. Whether you're a beginner or an advanced practitioner, DeepLearning.AI is your gateway to a smarter future.

-------

In [15]:
print(res.summary)

**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI is at the forefront of AI education, led by visionary founder Andrew Ng. This platform offers a variety of deep learning courses tailored for all levels, from beginners to advanced practitioners. **Courses Offered** cover essential topics like neural networks and natural language processing, empowering learners to tackle real-world challenges. 

Engage with a vibrant community, joining others in building skills and expertise. **Benefits of Learning AI** include increased job opportunities and the ability to innovate across industries. 

Discover your path in artificial intelligence training today—enroll now to unlock your potential! 

*Disclaimer: Course outcomes may vary. Please review all course details before enrollment.*
